In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Leemos la data
url=r'C:\Users\andre\Downloads\ObesityDataSet_raw_and_data_sinthetic.csv'
df=pd.read_csv(url)

In [ ]:
#Hacemos que se vean rodas las columnas
pd.set_option('display.max_columns', None)

FAVC:" Do you eat high caloric food frequently? "

FCVC:" Do you usually eat vegetables in your meals? "

NCP:" How many main meals do you have daily? "

CAEC:" Do you eat any food between meals? "

SMOKE:" Do you smoke? "

CH2O:" How much water do you drink daily? "

SCC:" Do you monitor the calories you eat daily? "

FAF:" How often do you have physical activity? "

TUE:" How much time do you use technological devices such as cell phone, videogames, television, computer and others? "

CALC :" How often do you drink alcohol? "

MTRANS:" Which transportation do you usually use?

In [ ]:
#Miramos tanto sus dtypes como si tiene nulos
df.info()

In [ ]:
#Miramos si tiene datos nulos
df.isnull().sum()

### **Limpieza**

In [ ]:
#Quitamos las mayusculas 
df.columns=df.columns.str.lower()

In [ ]:
#Cambiamos los nombres de dos columnas "family_history_with_overweight" y "nobeyesdad"
df=df.rename(columns={"family_history_with_overweight": "family_history", "nobeyesdad": "obesity_level"})

In [ ]:
#Calculo el índice de masa corporal (IMC) por nivel de obesidad
df['bmi'] = df['weight'] / (df['height'] / 100) ** 2

In [ ]:
#seleccionamos las columnas categoricas y numericas y las separamos en dataframes distintos.
potential_categorical_from_numerical = df.select_dtypes("number").loc[:, df.select_dtypes("number").nunique() < 20]

df_categorical = pd.concat([df.select_dtypes("object"), potential_categorical_from_numerical], axis=1)

df_numerical = df.select_dtypes("number").drop(columns=potential_categorical_from_numerical.columns)

In [ ]:
# Realizar One-Hot Encoding usando pd.get_dummies()
df_encoded = pd.get_dummies(df_categorical, columns=['calc', 'caec', 'mtrans'], drop_first=True)

In [ ]:
# Convertir 'gender' a formato binario
df_encoded['gender'] = df_encoded['gender'].apply(lambda x: 1 if x == 'Male' else 0)

# Asegurar que las columnas binarias estén en formato correcto (0 y 1)
binary_columns = ['favc', 'scc', 'smoke', 'family_history']
for col in binary_columns:
    df_encoded[col] = df_encoded[col].apply(lambda x: 1 if x == 'yes' else 0)